## Core

> The building blocks to the UI

In [ ]:
#| default_exp core

## Imports

In [ ]:
#| export
import fasthtml.common as fh
from monsterui.foundations import *
from fasthtml.common import FastHTML, fast_app
from enum import Enum, auto
from fastcore.all import *
import httpx
from pathlib import Path

In [ ]:
from fasthtml.jupyter import *
from functools import partial

In [ ]:
from nbdev.showdoc import *

## App

In [ ]:
#| export
@delegates(fh.fast_app, but=['pico'])
def fast_app(*args, pico=False, **kwargs):
    "Create a FastHTML or FastHTMLWithLiveReload app with `bg-background text-foreground` to bodykw for frankenui themes"
    if 'bodykw' not in kwargs: kwargs['bodykw'] = {}
    if 'class' not in kwargs['bodykw']: kwargs['bodykw']['class'] = ''
    kwargs['bodykw']['class'] = stringify((kwargs['bodykw']['class'],'bg-background text-foreground'))
    return fh.fast_app(*args, pico=pico, **kwargs)

In [ ]:
assert fast_app(bodykw={'something':'test'})[0].bodykw == {'something': 'test', 'class': ' bg-background text-foreground'}

In [ ]:
#| export
@delegates(fh.FastHTML, but=['pico'])
def FastHTML(*args, pico=False, **kwargs):
    "Create a FastHTML app and adds `bg-background text-foreground` to bodykw for frankenui themes"
    if 'bodykw' not in kwargs: kwargs['bodykw'] = {}
    if 'class' not in kwargs['bodykw']: kwargs['bodykw']['class'] = ''
    kwargs['bodykw']['class'] = stringify((kwargs['bodykw']['class'],'bg-background text-foreground'))
    bodykw = kwargs.pop('bodykw',{})
    return fh.FastHTML(*args, pico=pico, **bodykw, **kwargs)

In [ ]:
FastHTML(bodykw={'something':'test'}).bodykw

{'pico': False, 'something': 'test', 'class': ' bg-background text-foreground'}

## Theme / Headers

You can select a theme color to and get all the headers.

In [ ]:
#| export
class ThemeRadii(VEnum):
    none = 'uk-radii-none'
    sm = 'uk-radii-sm'
    md = 'uk-radii-md'
    lg = 'uk-radii-lg'

class ThemeShadows:
    none = 'uk-shadows-none'
    sm = 'uk-shadows-sm'
    md = 'uk-shadows-md'
    lg = 'uk-shadows-lg'

class ThemeFont:
    sm = 'uk-font-sm'
    default = 'uk-font-base'

In [ ]:
#| export
def _headers_theme(color, mode='auto', radii=ThemeRadii.sm, shadows=ThemeShadows.sm, font=ThemeFont.sm):
    return fh.Script(f'''
        const htmlElement = document.documentElement;
        const stored = JSON.parse(localStorage.getItem("__FRANKEN__") || "{{}}");
        const prefersDark = window.matchMedia("(prefers-color-scheme: dark)");
        const configMode = "{mode}" === "auto" ? "system" : "{mode}";
        let modeToUse = stored.mode || (prefersDark.matches ? "dark" : "light");
        if (configMode === "dark" || configMode === "light") modeToUse = configMode;
        if (configMode === "system" && stored.mode) modeToUse = stored.mode;
        const isDark = modeToUse === "dark";
        htmlElement.classList.toggle("dark", isDark);
        htmlElement.dataset.mode = isDark ? "dark" : "light";
        htmlElement.dataset.theme = "monster";
        const themeClass = stored.theme || "uk-theme-{color}";
        const radiiClass = stored.radii || "{radii}";
        const shadowClass = stored.shadows || "{shadows}";
        const fontClass = stored.font || "{font}";
        htmlElement.classList.add(themeClass, radiiClass, shadowClass, fontClass);
        htmlElement.dataset.depth = shadowClass.includes("none") ? "flat" : "raised";
    ''')


In [ ]:
#| export
HEADER_URLS = {
        'franken_css': "https://cdn.jsdelivr.net/npm/franken-ui@2.1.1/dist/css/core.min.css",
        'franken_js_core': "https://cdn.jsdelivr.net/npm/franken-ui@2.1.1/dist/js/core.iife.js",
        'franken_icons': "https://cdn.jsdelivr.net/npm/franken-ui@2.1.1/dist/js/icon.iife.js",
        'tailwind': "https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4",
        'daisyui': "https://cdn.jsdelivr.net/npm/daisyui@5",
        'apex_charts': "https://cdn.jsdelivr.net/npm/franken-ui@2.1.1/dist/js/chart.iife.js",
        'highlight_js': "https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/highlight.min.js",
        'highlight_python': "https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/languages/python.min.js",
        'highlight_light_css': "https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/styles/atom-one-light.css",
        'highlight_dark_css': "https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/styles/atom-one-dark.css",
        'highlight_copy': "https://cdn.jsdelivr.net/gh/arronhunt/highlightjs-copy/dist/highlightjs-copy.min.js",
        'highlight_copy_css': "https://cdn.jsdelivr.net/gh/arronhunt/highlightjs-copy/dist/highlightjs-copy.min.css",
}


def _download_resource(url, static_dir):
    "Download a single resource and return its local path"
    static = Path(static_dir)
    fname = static/f"{url[0]}.{'js' if 'js' in url[1] else 'css'}"
    content = httpx.get(url[1], follow_redirects=True).content
    fname.write_bytes(content)
    return (url[0], f"/{static_dir}/{fname.name}")


In [ ]:
#| export
THEME_VARIABLES = {'blue': {'dark': {'--accent': '217.2 32.6% 17.5%',
                   '--accent-foreground': '210 40% 98%',
                   '--background': '222.2 84% 4.9%',
                   '--border': '217.2 32.6% 17.5%',
                   '--card': '222.2 84% 4.9%',
                   '--card-foreground': '210 40% 98%',
                   '--destructive': '0 62.8% 30.6%',
                   '--destructive-foreground': '210 40% 98%',
                   '--foreground': '210 40% 98%',
                   '--input': '217.2 32.6% 17.5%',
                   '--muted': '217.2 32.6% 17.5%',
                   '--muted-foreground': '215 20.2% 65.1%',
                   '--popover': '222.2 84% 4.9%',
                   '--popover-foreground': '210 40% 98%',
                   '--primary': '217.2 91.2% 59.8%',
                   '--primary-foreground': '222.2 47.4% 11.2%',
                   '--ring': '224.3 76.3% 48%',
                   '--secondary': '217.2 32.6% 17.5%',
                   '--secondary-foreground': '210 40% 98%'},
          'light': {'--accent': '210 40% 96.1%',
                    '--accent-foreground': '222.2 47.4% 11.2%',
                    '--background': '0 0% 100%',
                    '--border': '214.3 31.8% 91.4%',
                    '--card': '0 0% 100%',
                    '--card-foreground': '222.2 84% 4.9%',
                    '--destructive': '0 84.2% 60.2%',
                    '--destructive-foreground': '210 40% 98%',
                    '--foreground': '222.2 84% 4.9%',
                    '--input': '214.3 31.8% 91.4%',
                    '--muted': '210 40% 96.1%',
                    '--muted-foreground': '215.4 16.3% 46.9%',
                    '--popover': '0 0% 100%',
                    '--popover-foreground': '222.2 84% 4.9%',
                    '--primary': '221.2 83.2% 53.3%',
                    '--primary-foreground': '210 40% 98%',
                    '--ring': '221.2 83.2% 53.3%',
                    '--secondary': '210 40% 96.1%',
                    '--secondary-foreground': '222.2 47.4% 11.2%'}},
 'gray': {'dark': {'--accent': '215 27.9% 16.9%',
                   '--accent-foreground': '210 20% 98%',
                   '--background': '224 71.4% 4.1%',
                   '--border': '215 27.9% 16.9%',
                   '--card': '224 71.4% 4.1%',
                   '--card-foreground': '210 20% 98%',
                   '--destructive': '0 62.8% 30.6%',
                   '--destructive-foreground': '210 20% 98%',
                   '--foreground': '210 20% 98%',
                   '--input': '215 27.9% 16.9%',
                   '--muted': '215 27.9% 16.9%',
                   '--muted-foreground': '217.9 10.6% 64.9%',
                   '--popover': '224 71.4% 4.1%',
                   '--popover-foreground': '210 20% 98%',
                   '--primary': '210 20% 98%',
                   '--primary-foreground': '220.9 39.3% 11%',
                   '--ring': '216 12.2% 83.9%',
                   '--secondary': '215 27.9% 16.9%',
                   '--secondary-foreground': '210 20% 98%'},
          'light': {'--accent': '220 14.3% 95.9%',
                    '--accent-foreground': '220.9 39.3% 11%',
                    '--background': '0 0% 100%',
                    '--border': '220 13% 91%',
                    '--card': '0 0% 100%',
                    '--card-foreground': '224 71.4% 4.1%',
                    '--destructive': '0 84.2% 60.2%',
                    '--destructive-foreground': '210 20% 98%',
                    '--foreground': '224 71.4% 4.1%',
                    '--input': '220 13% 91%',
                    '--muted': '220 14.3% 95.9%',
                    '--muted-foreground': '220 8.9% 46.1%',
                    '--popover': '0 0% 100%',
                    '--popover-foreground': '224 71.4% 4.1%',
                    '--primary': '220.9 39.3% 11%',
                    '--primary-foreground': '210 20% 98%',
                    '--ring': '224 71.4% 4.1%',
                    '--secondary': '220 14.3% 95.9%',
                    '--secondary-foreground': '220.9 39.3% 11%'}},
 'green': {'dark': {'--accent': '12 6.5% 15.1%',
                    '--accent-foreground': '0 0% 98%',
                    '--background': '20 14.3% 4.1%',
                    '--border': '240 3.7% 15.9%',
                    '--card': '24 9.8% 10%',
                    '--card-foreground': '0 0% 95%',
                    '--destructive': '0 62.8% 30.6%',
                    '--destructive-foreground': '0 85.7% 97.3%',
                    '--foreground': '0 0% 95%',
                    '--input': '240 3.7% 15.9%',
                    '--muted': '0 0% 15%',
                    '--muted-foreground': '240 5% 64.9%',
                    '--popover': '0 0% 9%',
                    '--popover-foreground': '0 0% 95%',
                    '--primary': '142.1 70.6% 45.3%',
                    '--primary-foreground': '144.9 80.4% 10%',
                    '--ring': '142.4 71.8% 29.2%',
                    '--secondary': '240 3.7% 15.9%',
                    '--secondary-foreground': '0 0% 98%'},
           'light': {'--accent': '240 4.8% 95.9%',
                     '--accent-foreground': '240 5.9% 10%',
                     '--background': '0 0% 100%',
                     '--border': '240 5.9% 90%',
                     '--card': '0 0% 100%',
                     '--card-foreground': '240 10% 3.9%',
                     '--destructive': '0 84.2% 60.2%',
                     '--destructive-foreground': '0 0% 98%',
                     '--foreground': '240 10% 3.9%',
                     '--input': '240 5.9% 90%',
                     '--muted': '240 4.8% 95.9%',
                     '--muted-foreground': '240 3.8% 46.1%',
                     '--popover': '0 0% 100%',
                     '--popover-foreground': '240 10% 3.9%',
                     '--primary': '142.1 76.2% 36.3%',
                     '--primary-foreground': '355.7 100% 97.3%',
                     '--ring': '142.1 76.2% 36.3%',
                     '--secondary': '240 4.8% 95.9%',
                     '--secondary-foreground': '240 5.9% 10%'}},
 'neutral': {'dark': {'--accent': '0 0% 14.9%',
                      '--accent-foreground': '0 0% 98%',
                      '--background': '0 0% 3.9%',
                      '--border': '0 0% 14.9%',
                      '--card': '0 0% 3.9%',
                      '--card-foreground': '0 0% 98%',
                      '--destructive': '0 62.8% 30.6%',
                      '--destructive-foreground': '0 0% 98%',
                      '--foreground': '0 0% 98%',
                      '--input': '0 0% 14.9%',
                      '--muted': '0 0% 14.9%',
                      '--muted-foreground': '0 0% 63.9%',
                      '--popover': '0 0% 3.9%',
                      '--popover-foreground': '0 0% 98%',
                      '--primary': '0 0% 98%',
                      '--primary-foreground': '0 0% 9%',
                      '--ring': '0 0% 83.1%',
                      '--secondary': '0 0% 14.9%',
                      '--secondary-foreground': '0 0% 98%'},
             'light': {'--accent': '0 0% 96.1%',
                       '--accent-foreground': '0 0% 9%',
                       '--background': '0 0% 100%',
                       '--border': '0 0% 89.8%',
                       '--card': '0 0% 100%',
                       '--card-foreground': '0 0% 3.9%',
                       '--destructive': '0 84.2% 60.2%',
                       '--destructive-foreground': '0 0% 98%',
                       '--foreground': '0 0% 3.9%',
                       '--input': '0 0% 89.8%',
                       '--muted': '0 0% 96.1%',
                       '--muted-foreground': '0 0% 45.1%',
                       '--popover': '0 0% 100%',
                       '--popover-foreground': '0 0% 3.9%',
                       '--primary': '0 0% 9%',
                       '--primary-foreground': '0 0% 98%',
                       '--ring': '0 0% 3.9%',
                       '--secondary': '0 0% 96.1%',
                       '--secondary-foreground': '0 0% 9%'}},
 'orange': {'dark': {'--accent': '12 6.5% 15.1%',
                     '--accent-foreground': '60 9.1% 97.8%',
                     '--background': '20 14.3% 4.1%',
                     '--border': '12 6.5% 15.1%',
                     '--card': '20 14.3% 4.1%',
                     '--card-foreground': '60 9.1% 97.8%',
                     '--destructive': '0 72.2% 50.6%',
                     '--destructive-foreground': '60 9.1% 97.8%',
                     '--foreground': '60 9.1% 97.8%',
                     '--input': '12 6.5% 15.1%',
                     '--muted': '12 6.5% 15.1%',
                     '--muted-foreground': '24 5.4% 63.9%',
                     '--popover': '20 14.3% 4.1%',
                     '--popover-foreground': '60 9.1% 97.8%',
                     '--primary': '20.5 90.2% 48.2%',
                     '--primary-foreground': '60 9.1% 97.8%',
                     '--ring': '20.5 90.2% 48.2%',
                     '--secondary': '12 6.5% 15.1%',
                     '--secondary-foreground': '60 9.1% 97.8%'},
            'light': {'--accent': '60 4.8% 95.9%',
                      '--accent-foreground': '24 9.8% 10%',
                      '--background': '0 0% 100%',
                      '--border': '20 5.9% 90%',
                      '--card': '0 0% 100%',
                      '--card-foreground': '20 14.3% 4.1%',
                      '--destructive': '0 84.2% 60.2%',
                      '--destructive-foreground': '60 9.1% 97.8%',
                      '--foreground': '20 14.3% 4.1%',
                      '--input': '20 5.9% 90%',
                      '--muted': '60 4.8% 95.9%',
                      '--muted-foreground': '25 5.3% 44.7%',
                      '--popover': '0 0% 100%',
                      '--popover-foreground': '20 14.3% 4.1%',
                      '--primary': '24.6 95% 53.1%',
                      '--primary-foreground': '60 9.1% 97.8%',
                      '--ring': '24.6 95% 53.1%',
                      '--secondary': '60 4.8% 95.9%',
                      '--secondary-foreground': '24 9.8% 10%'}},
 'red': {'dark': {'--accent': '0 0% 14.9%',
                  '--accent-foreground': '0 0% 98%',
                  '--background': '0 0% 3.9%',
                  '--border': '0 0% 14.9%',
                  '--card': '0 0% 3.9%',
                  '--card-foreground': '0 0% 98%',
                  '--destructive': '0 62.8% 30.6%',
                  '--destructive-foreground': '0 0% 98%',
                  '--foreground': '0 0% 98%',
                  '--input': '0 0% 14.9%',
                  '--muted': '0 0% 14.9%',
                  '--muted-foreground': '0 0% 63.9%',
                  '--popover': '0 0% 3.9%',
                  '--popover-foreground': '0 0% 98%',
                  '--primary': '0 72.2% 50.6%',
                  '--primary-foreground': '0 85.7% 97.3%',
                  '--ring': '0 72.2% 50.6%',
                  '--secondary': '0 0% 14.9%',
                  '--secondary-foreground': '0 0% 98%'},
         'light': {'--accent': '0 0% 96.1%',
                   '--accent-foreground': '0 0% 9%',
                   '--background': '0 0% 100%',
                   '--border': '0 0% 89.8%',
                   '--card': '0 0% 100%',
                   '--card-foreground': '0 0% 3.9%',
                   '--destructive': '0 84.2% 60.2%',
                   '--destructive-foreground': '0 0% 98%',
                   '--foreground': '0 0% 3.9%',
                   '--input': '0 0% 89.8%',
                   '--muted': '0 0% 96.1%',
                   '--muted-foreground': '0 0% 45.1%',
                   '--popover': '0 0% 100%',
                   '--popover-foreground': '0 0% 3.9%',
                   '--primary': '0 72.2% 50.6%',
                   '--primary-foreground': '0 85.7% 97.3%',
                   '--ring': '0 72.2% 50.6%',
                   '--secondary': '0 0% 96.1%',
                   '--secondary-foreground': '0 0% 9%'}},
 'rose': {'dark': {'--accent': '12 6.5% 15.1%',
                   '--accent-foreground': '0 0% 98%',
                   '--background': '20 14.3% 4.1%',
                   '--border': '240 3.7% 15.9%',
                   '--card': '24 9.8% 10%',
                   '--card-foreground': '0 0% 95%',
                   '--destructive': '0 62.8% 30.6%',
                   '--destructive-foreground': '0 85.7% 97.3%',
                   '--foreground': '0 0% 95%',
                   '--input': '240 3.7% 15.9%',
                   '--muted': '0 0% 15%',
                   '--muted-foreground': '240 5% 64.9%',
                   '--popover': '0 0% 9%',
                   '--popover-foreground': '0 0% 95%',
                   '--primary': '346.8 77.2% 49.8%',
                   '--primary-foreground': '355.7 100% 97.3%',
                   '--ring': '346.8 77.2% 49.8%',
                   '--secondary': '240 3.7% 15.9%',
                   '--secondary-foreground': '0 0% 98%'},
          'light': {'--accent': '240 4.8% 95.9%',
                    '--accent-foreground': '240 5.9% 10%',
                    '--background': '0 0% 100%',
                    '--border': '240 5.9% 90%',
                    '--card': '0 0% 100%',
                    '--card-foreground': '240 10% 3.9%',
                    '--destructive': '0 84.2% 60.2%',
                    '--destructive-foreground': '0 0% 98%',
                    '--foreground': '240 10% 3.9%',
                    '--input': '240 5.9% 90%',
                    '--muted': '240 4.8% 95.9%',
                    '--muted-foreground': '240 3.8% 46.1%',
                    '--popover': '0 0% 100%',
                    '--popover-foreground': '240 10% 3.9%',
                    '--primary': '346.8 77.2% 49.8%',
                    '--primary-foreground': '355.7 100% 97.3%',
                    '--ring': '346.8 77.2% 49.8%',
                    '--secondary': '240 4.8% 95.9%',
                    '--secondary-foreground': '240 5.9% 10%'}},
 'slate': {'dark': {'--accent': '217.2 32.6% 17.5%',
                    '--accent-foreground': '210 40% 98%',
                    '--background': '222.2 84% 4.9%',
                    '--border': '217.2 32.6% 17.5%',
                    '--card': '222.2 84% 4.9%',
                    '--card-foreground': '210 40% 98%',
                    '--destructive': '0 62.8% 30.6%',
                    '--destructive-foreground': '210 40% 98%',
                    '--foreground': '210 40% 98%',
                    '--input': '217.2 32.6% 17.5%',
                    '--muted': '217.2 32.6% 17.5%',
                    '--muted-foreground': '215 20.2% 65.1%',
                    '--popover': '222.2 84% 4.9%',
                    '--popover-foreground': '210 40% 98%',
                    '--primary': '210 40% 98%',
                    '--primary-foreground': '222.2 47.4% 11.2%',
                    '--ring': '212.7 26.8% 83.9',
                    '--secondary': '217.2 32.6% 17.5%',
                    '--secondary-foreground': '210 40% 98%'},
           'light': {'--accent': '210 40% 96.1%',
                     '--accent-foreground': '222.2 47.4% 11.2%',
                     '--background': '0 0% 100%',
                     '--border': '214.3 31.8% 91.4%',
                     '--card': '0 0% 100%',
                     '--card-foreground': '222.2 84% 4.9%',
                     '--destructive': '0 84.2% 60.2%',
                     '--destructive-foreground': '210 40% 98%',
                     '--foreground': '222.2 84% 4.9%',
                     '--input': '214.3 31.8% 91.4%',
                     '--muted': '210 40% 96.1%',
                     '--muted-foreground': '215.4 16.3% 46.9%',
                     '--popover': '0 0% 100%',
                     '--popover-foreground': '222.2 84% 4.9%',
                     '--primary': '222.2 47.4% 11.2%',
                     '--primary-foreground': '210 40% 98%',
                     '--ring': '222.2 84% 4.9%',
                     '--secondary': '210 40% 96.1%',
                     '--secondary-foreground': '222.2 47.4% 11.2%'}},
 'stone': {'dark': {'--accent': '12 6.5% 15.1%',
                    '--accent-foreground': '60 9.1% 97.8%',
                    '--background': '20 14.3% 4.1%',
                    '--border': '12 6.5% 15.1%',
                    '--card': '20 14.3% 4.1%',
                    '--card-foreground': '60 9.1% 97.8%',
                    '--destructive': '0 62.8% 30.6%',
                    '--destructive-foreground': '60 9.1% 97.8%',
                    '--foreground': '60 9.1% 97.8%',
                    '--input': '12 6.5% 15.1%',
                    '--muted': '12 6.5% 15.1%',
                    '--muted-foreground': '24 5.4% 63.9%',
                    '--popover': '20 14.3% 4.1%',
                    '--popover-foreground': '60 9.1% 97.8%',
                    '--primary': '60 9.1% 97.8%',
                    '--primary-foreground': '24 9.8% 10%',
                    '--ring': '24 5.7% 82.9%',
                    '--secondary': '12 6.5% 15.1%',
                    '--secondary-foreground': '60 9.1% 97.8%'},
           'light': {'--accent': '60 4.8% 95.9%',
                     '--accent-foreground': '24 9.8% 10%',
                     '--background': '0 0% 100%',
                     '--border': '20 5.9% 90%',
                     '--card': '0 0% 100%',
                     '--card-foreground': '20 14.3% 4.1%',
                     '--destructive': '0 84.2% 60.2%',
                     '--destructive-foreground': '60 9.1% 97.8%',
                     '--foreground': '20 14.3% 4.1%',
                     '--input': '20 5.9% 90%',
                     '--muted': '60 4.8% 95.9%',
                     '--muted-foreground': '25 5.3% 44.7%',
                     '--popover': '0 0% 100%',
                     '--popover-foreground': '20 14.3% 4.1%',
                     '--primary': '24 9.8% 10%',
                     '--primary-foreground': '60 9.1% 97.8%',
                     '--ring': '20 14.3% 4.1%',
                     '--secondary': '60 4.8% 95.9%',
                     '--secondary-foreground': '24 9.8% 10%'}},
 'violet': {'dark': {'--accent': '215 27.9% 16.9%',
                     '--accent-foreground': '210 20% 98%',
                     '--background': '224 71.4% 4.1%',
                     '--border': '215 27.9% 16.9%',
                     '--card': '224 71.4% 4.1%',
                     '--card-foreground': '210 20% 98%',
                     '--destructive': '0 62.8% 30.6%',
                     '--destructive-foreground': '210 20% 98%',
                     '--foreground': '210 20% 98%',
                     '--input': '215 27.9% 16.9%',
                     '--muted': '215 27.9% 16.9%',
                     '--muted-foreground': '217.9 10.6% 64.9%',
                     '--popover': '224 71.4% 4.1%',
                     '--popover-foreground': '210 20% 98%',
                     '--primary': '263.4 70% 50.4%',
                     '--primary-foreground': '210 20% 98%',
                     '--ring': '263.4 70% 50.4%',
                     '--secondary': '215 27.9% 16.9%',
                     '--secondary-foreground': '210 20% 98%'},
            'light': {'--accent': '220 14.3% 95.9%',
                      '--accent-foreground': '220.9 39.3% 11%',
                      '--background': '0 0% 100%',
                      '--border': '220 13% 91%',
                      '--card': '0 0% 100%',
                      '--card-foreground': '224 71.4% 4.1%',
                      '--destructive': '0 84.2% 60.2%',
                      '--destructive-foreground': '210 20% 98%',
                      '--foreground': '224 71.4% 4.1%',
                      '--input': '220 13% 91%',
                      '--muted': '220 14.3% 95.9%',
                      '--muted-foreground': '220 8.9% 46.1%',
                      '--popover': '0 0% 100%',
                      '--popover-foreground': '224 71.4% 4.1%',
                      '--primary': '262.1 83.3% 57.8%',
                      '--primary-foreground': '210 20% 98%',
                      '--ring': '262.1 83.3% 57.8%',
                      '--secondary': '220 14.3% 95.9%',
                      '--secondary-foreground': '220.9 39.3% 11%'}},
 'yellow': {'dark': {'--accent': '12 6.5% 15.1%',
                     '--accent-foreground': '60 9.1% 97.8%',
                     '--background': '20 14.3% 4.1%',
                     '--border': '12 6.5% 15.1%',
                     '--card': '20 14.3% 4.1%',
                     '--card-foreground': '60 9.1% 97.8%',
                     '--destructive': '0 62.8% 30.6%',
                     '--destructive-foreground': '60 9.1% 97.8%',
                     '--foreground': '60 9.1% 97.8%',
                     '--input': '12 6.5% 15.1%',
                     '--muted': '12 6.5% 15.1%',
                     '--muted-foreground': '24 5.4% 63.9%',
                     '--popover': '20 14.3% 4.1%',
                     '--popover-foreground': '60 9.1% 97.8%',
                     '--primary': '47.9 95.8% 53.1%',
                     '--primary-foreground': '26 83.3% 14.1%',
                     '--ring': '35.5 91.7% 32.9%',
                     '--secondary': '12 6.5% 15.1%',
                     '--secondary-foreground': '60 9.1% 97.8%'},
            'light': {'--accent': '60 4.8% 95.9%',
                      '--accent-foreground': '24 9.8% 10%',
                      '--background': '0 0% 100%',
                      '--border': '20 5.9% 90%',
                      '--card': '0 0% 100%',
                      '--card-foreground': '20 14.3% 4.1%',
                      '--destructive': '0 84.2% 60.2%',
                      '--destructive-foreground': '60 9.1% 97.8%',
                      '--foreground': '20 14.3% 4.1%',
                      '--input': '20 5.9% 90%',
                      '--muted': '60 4.8% 95.9%',
                      '--muted-foreground': '25 5.3% 44.7%',
                      '--popover': '0 0% 100%',
                      '--popover-foreground': '20 14.3% 4.1%',
                      '--primary': '47.9 95.8% 53.1%',
                      '--primary-foreground': '26 83.3% 14.1%',
                      '--ring': '20 14.3% 4.1%',
                      '--secondary': '60 4.8% 95.9%',
                      '--secondary-foreground': '24 9.8% 10%'}},
 'zinc': {'dark': {'--accent': '240 3.7% 15.9%',
                   '--accent-foreground': '0 0% 98%',
                   '--background': '240 10% 3.9%',
                   '--border': '240 3.7% 15.9%',
                   '--card': '240 10% 3.9%',
                   '--card-foreground': '0 0% 98%',
                   '--destructive': '0 62.8% 30.6%',
                   '--destructive-foreground': '0 0% 98%',
                   '--foreground': '0 0% 98%',
                   '--input': '240 3.7% 15.9%',
                   '--muted': '240 3.7% 15.9%',
                   '--muted-foreground': '240 5% 64.9%',
                   '--popover': '240 10% 3.9%',
                   '--popover-foreground': '0 0% 98%',
                   '--primary': '0 0% 98%',
                   '--primary-foreground': '240 5.9% 10%',
                   '--ring': '240 4.9% 83.9%',
                   '--secondary': '240 3.7% 15.9%',
                   '--secondary-foreground': '0 0% 98%'},
          'light': {'--accent': '240 4.8% 95.9%',
                    '--accent-foreground': '240 5.9% 10%',
                    '--background': '0 0% 100%',
                    '--border': '240 5.9% 90%',
                    '--card': '0 0% 100%',
                    '--card-foreground': '240 10% 3.9%',
                    '--destructive': '0 84.2% 60.2%',
                    '--destructive-foreground': '0 0% 98%',
                    '--foreground': '240 10% 3.9%',
                    '--input': '240 5.9% 90%',
                    '--muted': '240 4.8% 95.9%',
                    '--muted-foreground': '240 3.8% 46.1%',
                    '--popover': '0 0% 100%',
                    '--popover-foreground': '240 10% 3.9%',
                    '--primary': '240 5.9% 10%',
                    '--primary-foreground': '0 0% 98%',
                    '--ring': '240 5.9% 10%',
                    '--secondary': '240 4.8% 95.9%',
                    '--secondary-foreground': '240 5.9% 10%'}}}

def _build_monster_theme_css():
    rules = []
    for theme, modes in THEME_VARIABLES.items():
        for mode, values in modes.items():
            if not values:
                continue
            mode_cls = ".dark" if mode == "dark" else ""
            selector = f':root[data-theme="monster"]{mode_cls}.uk-theme-{theme}'
            props = '\n  '.join(f'{k}: {v};' for k, v in values.items())
            rules.append(f'{selector} {{\n  {props}\n}}')
    return "\n".join(rules)

monster_theme_styles = Style(_build_monster_theme_css())
for modes in THEME_VARIABLES.values():
    if 'light' in modes:
        modes['light'].setdefault('--border-alpha', '0.3')
        modes['light'].setdefault('--input-alpha', '0.4')
    if 'dark' in modes:
        modes['dark'].setdefault('--border-alpha', '0.75')
        modes['dark'].setdefault('--input-alpha', '0.75')



In [ ]:
#| export
tailwind_theme_styles = Style("""
@theme {
  --font-sans: var(--uk-global-font-family-sans, ui-sans-serif, system-ui, sans-serif);
  --font-mono: var(--uk-global-font-family-mono, ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", "Courier New");

  --color-background: hsl(var(--background));
  --color-foreground: hsl(var(--foreground));
  --color-muted: hsl(var(--muted));
  --color-muted-foreground: hsl(var(--muted-foreground));
  --color-popover: hsl(var(--popover));
  --color-popover-foreground: hsl(var(--popover-foreground));
  --color-card: hsl(var(--card));
  --color-card-foreground: hsl(var(--card-foreground));
  --color-border: hsl(var(--border));
  --color-input: hsl(var(--input));
  --color-primary: hsl(var(--primary));
  --color-primary-foreground: hsl(var(--primary-foreground));
  --color-secondary: hsl(var(--secondary));
  --color-secondary-foreground: hsl(var(--secondary-foreground));
  --color-accent: hsl(var(--accent));
  --color-accent-foreground: hsl(var(--accent-foreground));
  --color-destructive: hsl(var(--destructive));
  --color-destructive-foreground: hsl(var(--destructive-foreground));
  --color-ring: hsl(var(--ring));
  --color-chart-1: hsl(var(--chart-1));
  --color-chart-2: hsl(var(--chart-2));
  --color-chart-3: hsl(var(--chart-3));
  --color-chart-4: hsl(var(--chart-4));
  --color-chart-5: hsl(var(--chart-5));

  --radius-sm: calc(var(--radius, 0.5rem) / 2);
  --radius-md: var(--radius, 0.5rem);
  --radius-lg: calc(var(--radius, 0.5rem) * 1.5);
  --radius-xl: calc(var(--radius, 0.5rem) * 2);

  --shadow-sm: var(--uk-global-shadow-s, 0 1px 2px 0 rgb(0 0 0 / 0.05));
  --shadow-md: var(--uk-global-shadow, 0 4px 6px -1px rgb(0 0 0 / 0.1));
  --shadow-lg: var(--uk-global-shadow, 0 10px 15px -3px rgb(0 0 0 / 0.1));
}
""", type="text/tailwindcss")


In [ ]:
#| export
form_control_styles = Style("""
:root[data-theme="monster"] .uk-input {
  border: 1px solid hsl(var(--border) / var(--border-alpha, 1));
  background-color: hsl(var(--input) / var(--input-alpha, 1));
  color: hsl(var(--foreground));
}

:root[data-theme="monster"] .uk-input::placeholder {
  color: color-mix(in oklab, hsl(var(--foreground)) 60%, transparent);
}

:root[data-theme="monster"] .uk-input:focus {
  border-color: hsl(var(--ring));
  box-shadow: 0 0 0 1px hsl(var(--ring) / 0.35);
}

:root[data-theme="monster"] .uk-input-tag {
  border: 0 !important;
  background-color: hsl(var(--input) / var(--input-alpha, 1));
  box-shadow: 0 0 0 1px hsl(var(--border) / var(--border-alpha, 1));
  transition: box-shadow 0.2s ease;
}

:root[data-theme="monster"] .uk-input-tag input {
  border: 0;
  background-color: transparent;
  color: hsl(var(--foreground));
}

:root[data-theme="monster"] .uk-input-tag input::placeholder {
  color: color-mix(in oklab, hsl(var(--foreground)) 60%, transparent);
}

:root[data-theme="monster"] .uk-input-tag input:focus {
  box-shadow: none;
}

:root[data-theme="monster"] .uk-input-tag:focus-within {
  box-shadow: 0 0 0 1px hsl(var(--ring));
}

:root[data-theme="monster"] .uk-toggle-switch {
  border: 1px solid hsl(var(--input) / var(--input-alpha, 1));
  background-color: hsl(var(--input) / var(--input-alpha, 1));
  transition: background-color 0.2s ease, border-color 0.2s ease;
}

:root[data-theme="monster"] .uk-toggle-switch:checked {
  background-color: hsl(var(--primary));
  border-color: hsl(var(--primary));
}

:root[data-theme="monster"] .uk-toggle-switch::after {
  background-color: hsl(var(--color-base-100, var(--background)));
}

:root[data-theme="monster"] .steps .step.step-primary {
  --step-bg: hsl(var(--ring));
  --step-fg: hsl(var(--foreground));
}

:root[data-theme="monster"][data-mode="dark"] .steps .step.step-primary {
  --step-fg: hsl(var(--background));
}
""")


In [ ]:
#| export
daisy_styles = Style("""
:root[data-theme="monster"],
[data-theme="monster"] {
  --color-base-100: hsl(var(--background));
  --color-base-200: hsl(var(--card));
  --color-base-300: hsl(var(--muted));
  --color-base-content: hsl(var(--foreground));
  --color-primary: hsl(var(--primary));
  --color-primary-content: hsl(var(--primary-foreground));
  --color-secondary: hsl(var(--secondary));
  --color-secondary-content: hsl(var(--secondary-foreground));
  --color-accent: hsl(var(--accent));
  --color-accent-content: hsl(var(--accent-foreground));
  --color-neutral: hsl(var(--muted));
  --color-neutral-content: hsl(var(--muted-foreground));
  --color-info: oklch(74% 0.16 232.661);
  --color-info-content: oklch(29% 0.066 243.157);
  --color-success: oklch(76% 0.177 163.223);
  --color-success-content: oklch(37% 0.077 168.94);
  --color-warning: oklch(82% 0.189 84.429);
  --color-warning-content: oklch(41% 0.112 45.904);
  --color-error: oklch(71% 0.194 13.428);
  --color-error-content: oklch(27% 0.105 12.094);
  --color-ring: hsl(var(--ring));
  --radius-selector: var(--radius, 0.5rem);
  --radius-field: calc(var(--radius, 0.5rem) / 1.5);
  --radius-box: calc(var(--radius, 0.5rem) * 1.5);
  --size-selector: 0.25rem;
  --size-field: 0.25rem;
  --depth: 1;
  --noise: 0;
  --border-alpha: 0.15;
  --input-alpha: 0.2;
}

[data-theme="monster"][data-mode="dark"] {
  color-scheme: dark;
  --border-alpha: 0.6;
  --input-alpha: 0.6;
}

[data-theme="monster"][data-mode="light"] {
  color-scheme: light;
}

[data-theme="monster"][data-depth="flat"] {
  --depth: 0;
}

[data-theme="monster"][data-depth="raised"] {
  --depth: 1;
}
""")


In [ ]:
#| export
scrollspy_style= Style('''
.monster-navbar.navbar-bold a {
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
}
.monster-navbar.navbar-bold a.uk-active {
    transform: scale(1.15) ;
    font-weight: bold;
    text-shadow: 0 0 12px rgba(var(--p-rgb), 0.4);
    letter-spacing: 0.02em;
    color: hsl(var(--p) / 1);
}
.monster-navbar.navbar-underline a.uk-active { position: relative; }
.monster-navbar.navbar-underline a.uk-active::after {
    content: '';
    position: absolute;
    left: 0;
    bottom: -2px;
    width: 100%;
    height: 2px;
    background: currentColor;
    animation: slideIn 0.3s ease forwards;
}
@keyframes slideIn {
    from { transform: scaleX(0); }
    to { transform: scaleX(1); }
}
''')

In [ ]:
#| export
class Theme(Enum):
    "Selector to choose theme and get all headers needed for app.  Includes frankenui + tailwind + daisyui + highlight.js options"
    def _generate_next_value_(name, start, count, last_values): return name
    slate = auto()
    stone = auto()
    gray = auto()
    neutral = auto()
    red = auto()
    rose = auto()
    orange = auto()
    green = auto()
    blue = auto()
    yellow = auto()
    violet = auto()
    zinc = auto()

    def _create_headers(self, urls, mode='auto', icons=True, daisy=True, highlightjs=False, katex=False, apex_charts=False, radii=ThemeRadii.sm, shadows=ThemeShadows.sm, font=ThemeFont.sm):
        "Create header elements with given URLs"
        hdrs = [
            fh.Link(rel="stylesheet", href=urls['franken_css']),
            monster_theme_styles,
            fh.Script(type="module", src=urls['franken_js_core']),
            fh.Script(src=urls['tailwind']),
            tailwind_theme_styles,
            _headers_theme(self.value, mode=mode, radii=radii, shadows=shadows, font=font),
            scrollspy_style]

        if icons: hdrs.append(fh.Script(type="module", src=urls['franken_icons']))
        if daisy: hdrs += [fh.Link(rel="stylesheet", href=urls['daisyui']), daisy_styles]
        if apex_charts: hdrs += [fh.Script(type='module', src=urls['apex_charts'])]

        if highlightjs:
            hdrs += [
                fh.Script(src=urls['highlight_js']),
                fh.Script(src=urls['highlight_python']),
                fh.Link(rel="stylesheet", href=urls['highlight_light_css'], id='hljs-light'),
                fh.Link(rel="stylesheet", href=urls['highlight_dark_css'], id='hljs-dark'),
                fh.Script(src=urls['highlight_copy']),
                fh.Link(rel="stylesheet", href=urls['highlight_copy_css']),
                fh.Script('''
                    hljs.addPlugin(new CopyButtonPlugin());
                    hljs.configure({
                        cssSelector: 'pre code',
                        languages: ['python'],
                        ignoreUnescapedHTML: true
                    });
                    function updateTheme() {
                        const isDark = document.documentElement.classList.contains('dark');
                        document.getElementById('hljs-dark').disabled = !isDark;
                        document.getElementById('hljs-light').disabled = isDark;
                    }
                    new MutationObserver(mutations =>
                        mutations.forEach(m => m.target.tagName === 'HTML' &&
                            m.attributeName === 'class' && updateTheme())
                    ).observe(document.documentElement, { attributes: true });
                    updateTheme();
                    htmx.onLoad(hljs.highlightAll);
                ''', type='module'),
            ]

        if katex:
            hdrs += [
                fh.Link(rel="stylesheet",
                        href="https://cdn.jsdelivr.net/npm/katex@0.16.21/dist/katex.min.css"),
                fh.Script("""
                import katex from 'https://cdn.jsdelivr.net/npm/katex@0.16.21/dist/katex.mjs';
                import autoRender from 'https://cdn.jsdelivr.net/npm/katex@0.16.21/dist/contrib/auto-render.mjs';
                const options = {
                  delimiters: [
                    {left: '$$', right: '$$', display: true},
                    {left: '$', right: '$', display: false}
                  ],
                  ignoredClasses: ['nomath']
                };

                document.addEventListener('htmx:load', evt => {
                  const element = evt.detail.elt || document.body;
                  autoRender(element,options);
                });
                """,type="module"),
                ]
        return hdrs

    def headers(self, mode='auto', icons=True, daisy=True, highlightjs=False, katex=False, apex_charts=False, radii=ThemeRadii.sm, shadows=ThemeShadows.sm, font=ThemeFont.sm ):
        "Create frankenui and tailwind cdns"
        return self._create_headers(HEADER_URLS, mode=mode, icons=icons, daisy=daisy, highlightjs=highlightjs, katex=katex, apex_charts=apex_charts, radii=radii, shadows=shadows, font=font)    

    def local_headers(self, mode='auto', static_dir='static', icons=True, daisy=True, highlightjs=False, katex=False, apex_charts=False, radii='md', shadows='sm', font='sm'):
        "Create headers using local files downloaded from CDNs"
        Path(static_dir).mkdir(exist_ok=True)
        local_urls = dict([_download_resource(url, static_dir) for url in HEADER_URLS.items()])
        return self._create_headers(local_urls, mode=mode, icons=icons, daisy=daisy, highlightjs=highlightjs, katex=katex, apex_charts=apex_charts, radii=radii, shadows=shadows, font=font)


In [ ]:
for h in Theme.blue.headers(): print(h.href or h.src or "inline")

https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/css/core.min.css
https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/js/core.iife.js
https://cdn.tailwindcss.com/3.4.16
inline
inline
inline
https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/js/icon.iife.js
https://cdn.jsdelivr.net/npm/daisyui@4.12.24/dist/full.min.css
inline


`katex` and `highlightjs` are not included by default. To include them set `katex=True` or `highlightjs=True` when calling `.headers`.

In [ ]:
for h in Theme.blue.headers(katex=True, highlightjs=True): print(h.href or h.src or "inline")

https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/css/core.min.css
https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/js/core.iife.js
https://cdn.tailwindcss.com/3.4.16
inline
inline
inline
https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/js/icon.iife.js
https://cdn.jsdelivr.net/npm/daisyui@4.12.24/dist/full.min.css
inline
https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/highlight.min.js
https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/languages/python.min.js
https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/styles/atom-one-light.css
https://cdn.jsdelivr.net/gh/highlightjs/cdn-release@11.9.0/build/styles/atom-one-dark.css
https://cdn.jsdelivr.net/gh/arronhunt/highlightjs-copy/dist/highlightjs-copy.min.js
https://cdn.jsdelivr.net/gh/arronhunt/highlightjs-copy/dist/highlightjs-copy.min.css
inline
https://cdn.jsdelivr.net/npm/katex@0.16.21/dist/katex.min.css
inline


In [ ]:
app = FastHTML(hdrs=Theme.blue.headers())

In [ ]:
app, rt = fast_app(hdrs=Theme.blue.headers())
Show = partial(HTMX, app=app)

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()